# Economic Experiment

Add description of the experiment here

## Import Statements

In [13]:
from dfg_rating.model import factory
from dfg_rating.model.betting.betting import FixedBetting, BaseBetting
from dfg_rating.model.bookmaker.base_bookmaker import BaseBookmaker
from dfg_rating.model.evaluators.accuracy import RankProbabilityScore
from dfg_rating.model.evaluators.accuracy import Likelihood
from dfg_rating.model.evaluators.profitability import BettingReturnsEvaluator
from dfg_rating.model.forecast.base_forecast import SimpleForecast, BaseForecast
from dfg_rating.model.forecast.true_forecast import LogFunctionForecast
from dfg_rating.model.network.base_network import BaseNetwork
from dfg_rating.model.rating.controlled_trend_rating import ControlledTrendRating, ControlledRandomFunction

#specify rating error
from dfg_rating.model.rating.base_rating import RatingFunctionError

import networkx as nx
import pandas as pd
import numpy as np
import time

from dfg_rating.model.network.simple_network import RoundRobinNetwork

pd.set_option('display.max_columns', None)
pd.options.display.width = None
pd.set_option('display.float_format', lambda x: '%.5f' % x)

## External actors

We create a bookmaker and a betting strategy that are going to interact with the generated networks.

In [5]:
bookmaker: BaseBookmaker = factory.new_bookmaker(
    'simple',
    error=factory.new_forecast_error(error_type='factor', error=0.0, scope='positive'),
    margin=factory.new_bookmaker_margin('simple', margin=0.10)
)

In [6]:
betting = FixedBetting(100)

We also create an error rating that we are going to use for the calculated forecasts

In [4]:
rating_error = RatingFunctionError(error='normal', loc=0, scale=50)

## Generation of networks

Configure the number of leagues for the experiment:

In [7]:
numberLeagues = 5

For every league, we generate the schedule of matches and true dimensions and we add odds for each game.

In [14]:
networks = []
gen_start_time = time.time()
for league in range(numberLeagues):
    print(f"League : {league}")
    network: BaseNetwork = factory.new_network(
        'multiple-round-robin',
        teams=18,
        days_between_rounds=7,
        seasons=10,
        league_teams=18,
        league_promotion=0,
        create=True,
        true_forecast=LogFunctionForecast(
            outcomes=['home', 'draw', 'away'], 
            coefficients = [-0.9,0.3], 
            beta_parameter=0.006
        ),
        true_rating=ControlledTrendRating(
            starting_point=ControlledRandomFunction(distribution='normal', loc=1000, scale=100),
            delta=ControlledRandomFunction(distribution='normal', loc=0, scale=3),
            trend=ControlledRandomFunction(distribution='normal', loc=0, scale=20/365),
            season_delta=ControlledRandomFunction(distribution='normal', loc=0, scale=10)
        )
    )
    #adding biased bookmaker forecast to the network
    network.add_forecast(
        forecast=LogFunctionForecast(
            outcomes=['home', 'draw', 'away'], 
            coefficients=[-1.2, 0.0], 
            beta_parameter=0.006, 
            home_team_error=rating_error, 
            away_team_error=rating_error
        ),
        forecast_name='bookmaker_forecast',
        base_ranking='true_rating'
    )
    #adding odds    
    network.add_odds(
        bookmaker_name="bm",
        bookmaker=bookmaker,
        base_forecast='bookmaker_forecast'
    )

networks.append(network)
print(f"{numberLeagues} leagues added in {float(time.time() - gen_start_time)} seconds.")

League : 0
Season 0
Relegation candidates {9: 6.0, 13: 7.0, 0: 10.0, 11: 14.0, 1: 16.0, 14: 16.0}
Relegation teams []
Season 1
Relegation candidates {13: 11.0, 2: 13.0, 14: 13.0, 5: 13.0, 9: 14.0, 1: 14.0}
Relegation teams []
Season 2
Relegation candidates {13: 4.0, 0: 9.0, 4: 13.0, 1: 13.0, 12: 15.0, 2: 17.0}
Relegation teams []
Season 3
Relegation candidates {2: 5.0, 0: 11.0, 13: 11.0, 14: 12.0, 9: 15.0, 1: 18.0}
Relegation teams []
Season 4
Relegation candidates {7: 8.0, 1: 12.0, 9: 14.0, 12: 14.0, 4: 15.0, 14: 15.0}
Relegation teams []
Season 5
Relegation candidates {5: 4.0, 7: 12.0, 2: 13.0, 4: 14.0, 14: 14.0, 13: 15.0}
Relegation teams []
Season 6
Relegation candidates {5: 6.0, 9: 13.0, 2: 13.0, 12: 14.0, 0: 18.0, 14: 18.0}
Relegation teams []
Season 7
Relegation candidates {2: 9.0, 5: 10.0, 7: 11.0, 4: 12.0, 9: 12.0, 12: 19.0}
Relegation teams []
Season 8
Relegation candidates {2: 6.0, 9: 10.0, 5: 10.0, 0: 15.0, 12: 15.0, 1: 17.0}
Relegation teams []
Season 9
Relegation candidat

## Main loop

We iterate over the grid of possible parameters for the experiment and aggregate the final results.

In [9]:
def aggregate_measures(list_of_matches, prefix=''):
    return {prefix+'_rps': np.mean([m[3]['metrics']['rps'] for m in list_of_matches]), 
           prefix+'_likelihood': sum([m[3]['metrics']['likelihood'] for m in list_of_matches]), 
           prefix+'_betting_returns': sum([sum([b[0] for b in m[3]['metrics']['betting_returns']]) for m in list_of_matches]),
           prefix+'_betting_returns_expected': sum([sum(b[1] for b in m[3]['metrics']['betting_returns']) for m in list_of_matches])}

In [16]:
result_list = []
experiment_start_time = time.time()
#for c0 in np.arange(-1.0, 0.6, 0.1):
#    for c1 in np.arange(c0+0.2, 0.8, 0.1):
#        for beta in np.arange(0.004, 0.010, 0.002):
for c0 in np.arange(-1.0, -0.6, 0.1):
    for c1 in np.arange(c0+0.2, 0.0, 0.1):
        for beta in np.arange(0.004, 0.010, 0.002):
            
            print(f'Variables: c0: {c0}, c1: {c1} and beta: {beta}')
            cell_start_time = time.time()
            all_matches =  []
            is_matches =  []
            oos_matches =  []
            
            for network in networks:
                rps = RankProbabilityScore(outcomes=['home', 'draw', 'away'], forecast_name = 'player_forecast'+'_'+str(c0)+'_'+str(c1)+'_'+str(beta))
                betting_returns = BettingReturnsEvaluator(outcomes=['home', 'draw', 'away'], player_name = 'b', true_model = 'true_forecast', bookmaker_name = 'bm')
                likelihood = Likelihood(outcomes=['home', 'draw', 'away'], forecast_name = 'player_forecast'+'_'+str(c0)+'_'+str(c1)+'_'+str(beta))
            
                network.add_forecast(
                forecast=LogFunctionForecast(outcomes=['home', 'draw', 'away'], coefficients=[c0, c1], beta_parameter=beta, home_team_error=rating_error, away_team_error=rating_error),
                forecast_name='player_forecast'+'_'+str(c0)+'_'+str(c1)+'_'+str(beta),
                base_ranking='true_rating'
                )
                
                network.add_bets(
                bettor_name='b',
                bookmaker='bm',
                betting=betting,
                base_forecast='player_forecast'+'_'+str(c0)+'_'+str(c1)+'_'+str(beta)
                )
            
                network.add_evaluation(rps, 'rps')
                network.add_evaluation(betting_returns, 'betting_returns')
                network.add_evaluation(likelihood, 'likelihood')
                #network.serialize_network('economic_network')
                
                all_matches += [(a,h, match_id, match_attributes) for a,h, match_id, match_attributes in network.iterate_over_games()]
                is_matches += [(a,h, match_id, match_attributes) for a,h, match_id, match_attributes in network.iterate_over_games() if match_attributes['season'] < 4]
                oos_matches += [(a,h, match_id, match_attributes) for a,h, match_id, match_attributes in network.iterate_over_games() if match_attributes['season'] > 3]
                
            result_all = aggregate_measures(all_matches, 'all')
            result_is = aggregate_measures(is_matches, 'is')
            result_oos = aggregate_measures(oos_matches, 'oos')
            result = {
                'c0' : c0, 'c1' : c1, 'beta' : beta, **result_is, **result_oos, **result_all
            }
            
            result_list.append(result)
            print(f"Finished in {float(time.time() - cell_start_time)} seconds")
            
                 
            
#network.export(forecasts = ['bookmaker_forecast', 'true_forecast', 'player_forecast'+'_'+str(c0)+'_'+str(c1)+'_'+str(beta)],
#                      metrics = ['rps', 'betting_returns', 'loglikelihood'],
#                      odds=['bm'])
            
            
print(f"Experiment finished in {float(time.time() - experiment_start_time)} seconds with {len(result_list)} observations.")
            

Variables: c0: -1.0, c1: -0.8 and beta: 0.004
Finished in 1.0951571464538574 seconds
Variables: c0: -1.0, c1: -0.8 and beta: 0.006
Finished in 0.5940515995025635 seconds
Variables: c0: -1.0, c1: -0.8 and beta: 0.008
Finished in 0.5932610034942627 seconds
Variables: c0: -1.0, c1: -0.7000000000000001 and beta: 0.004
Finished in 0.5860798358917236 seconds
Variables: c0: -1.0, c1: -0.7000000000000001 and beta: 0.006
Finished in 0.6333410739898682 seconds
Variables: c0: -1.0, c1: -0.7000000000000001 and beta: 0.008
Finished in 0.6073160171508789 seconds
Variables: c0: -1.0, c1: -0.6000000000000001 and beta: 0.004
Finished in 0.5810320377349854 seconds
Variables: c0: -1.0, c1: -0.6000000000000001 and beta: 0.006
Finished in 0.5804049968719482 seconds
Variables: c0: -1.0, c1: -0.6000000000000001 and beta: 0.008
Finished in 0.7515444755554199 seconds
Variables: c0: -1.0, c1: -0.5000000000000001 and beta: 0.004
Finished in 0.5708787441253662 seconds
Variables: c0: -1.0, c1: -0.5000000000000001 

In [17]:
df = pd.DataFrame(result_list)
df.to_excel("Results_Economics.xlsx")

In [ ]:
for a,h,m_id,attributes in network.iterate_over_games():
    print([v for v in attributes['metrics']['betting_returns']])